In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
raw_data_path=os.path.join('data','raw')
df=pd.read_csv(os.path.join(raw_data_path,'fer2013.csv'))
X=[]
for i in range(len(df)):
    X.append(df.loc[i,'pixels'].split(' '))
X=np.array(X,np.float32)/255.
y=df.emotion.values
X_train=X[np.where(df.Usage=='Training')]
y_train=y[np.where((df.Usage=='Training'))]
X_test=X[np.where((df.Usage!='Training'))]
y_test=y[np.where((df.Usage!='Training'))]
def balance_classes(X,y):
    X1=X[np.where(y!=1)]
    X2=X[np.where(y==1)]
    X2=np.repeat(X2,9,axis=0)
    X=np.concatenate((X1,X2),axis=0)
    y1=y[np.where(y!=1)]
    y2=y[np.where(y==1)]
    y2=np.repeat(y2,9,axis=0)
    y=np.concatenate((y1,y2),axis=0)
    i=[i for i in range(len(y))]
    np.random.shuffle(i)
    return X[i],y[i]
X_train,y_train=balance_classes(X_train,y_train)

In [2]:
def train_test_split(X,y,size=0.8,rs=None):
    N=len(X)
    np.random.seed(rs)
    tr_i=np.random.choice(N,int(size*N),replace=False)
    t_i=[i for i in range(N) if i not in tr_i]
    X_train=X[tr_i]
    y_train=y[tr_i]
    X_test=X[t_i]
    y_test=y[t_i]
    return X_train,y_train,X_test,y_test
# X_train,y_train,X_test,y_test=train_test_split(X_s,y,size=0.9,rs=20)

In [3]:
import theano
from theano import tensor as T
# import theano.gpuarray as gp
import os
os.environ['THEANO_FLAGS'] = 'contexts=dev0->cuda0,mode=FAST_RUN,floatX=float32,exception_verbosity=high,dnn.conv.algo_fwd=time_once,dnn.conv.algo_bwd_filter=time_once,dnn.conv.algo_bwd_data=time_once,optimizer=fast_run,optimizer_including=unsafe'
# gp.use(None)

In [4]:
class HiddenLayer(object):
    def __init__(self,M1,M2,i):
        self.id=i
        self.M1=M1
        self.M2=M2
        W=np.random.randn(M1,M2)/np.sqrt(M1)
        b=np.random.randn(M2)
        self.W=theano.shared(W,name='W{0}'.format(i))
        self.b=theano.shared(b,name='b{0}'.format(i))
        self.params=[self.W,self.b]
    def forward(self,X,activation='relu'):
        if activation=='tanh':
            return T.tanh(X.dot(self.W)+self.b)
        elif activation=='sigmoid':
            return T.nnet.sigmoid(X.dot(self.W)+self.b)
        else:
            return T.nnet.relu(X.dot(self.W)+self.b)
    

In [8]:
class ANN(object):
    def __init__(self,hidden_layer_size):
        self.hidden_layer_size=hidden_layer_size
    def forward(self,X):
        z=X
        for h in self.hidden_layers:
            z=h.forward(z)
        return T.nnet.softmax(z.dot(self.W)+self.b)
    def predict(self,X):
        p=self.forward(X)
        return T.argmax(p,axis=1)
    def fit(self,X,y,lr=0.001,epochs=1000,mu=0.99,show_fig=True):
        X_t,y_t,X_v,y_v=train_test_split(X,y,size=0.9,rs=100)
        c_t=[]
        c_v=[]
        cl_t=[]
        cl_v=[]
        N,D=X_t.shape
        self.hidden_layers=[]
        M1=D
        cnt=0
        for M2 in self.hidden_layer_size:
            h=HiddenLayer(M1,M2,cnt)
            self.hidden_layers.append(h)
            M1=M2
            cnt+=1
        M2=len(set(y_t))
        W=np.random.randn(M1,M2)/np.sqrt(M1)
        b=np.random.randn(M2)
        self.W=theano.shared(W,name='W_f')
        self.b=theano.shared(b,name='b_f')
        self.params=[self.W,self.b]
        for h in self.hidden_layers:
            self.params+=h.params
        thX=T.matrix('X')
        thY=T.ivector('y')
        y_p=self.forward(thX)
        cost=-T.mean(T.log(y_p[T.arange(thY.shape[0]),thY]))
        preds=self.predict(thX)
        cost_preds=theano.function(inputs=[thX,thY],outputs=[cost,preds],allow_input_downcast=True)
#         updates=[(p,(p-lr*T.grad(cost,p)) )for p in self.params]
        grads=T.grad(cost,self.params)
        vparams=[theano.shared(np.zeros_like(p.get_value())) for p in self.params]
        updates=[(p,p+mu*v-lr*g) for p,g,v in zip(self.params,grads,vparams)]
        train=theano.function([thX,thY],updates=updates,allow_input_downcast=True)
        for i in range(epochs):
            c,p=cost_preds(X_v,y_v)
            c_v.append(c)
            cl_v.append(np.mean(p==y_v))
            c,p=cost_preds(X_t,y_t)
            c_t.append(c)
            cl_t.append(np.mean(p==y_t))
            train(X_t,y_t)
            if i %10==0:
                print('Epoch {0} Train C: {1} Cl: {2} Test C: {3} Cl:{4}'.format(i,c_t[i],cl_t[i],c_v[i],cl_v[i]))
#                 print('Epoch {0} Train C: {1} Test C: {2}'.format(i,c_t[i],c_v[i]))

            
        
        

In [9]:
a=ANN((10,20,30,30))

In [13]:
a.fit(X_train,y_train,lr=0.01,epochs=10000)

Epoch 0 Train C: 2.2847343798152178 Cl: 0.09811229595886392 Test C: 2.27947399866859 Cl:0.10186335403726708
Epoch 10 Train C: 2.084139366043242 Cl: 0.09811229595886392 Test C: 2.0832119423272233 Cl:0.10186335403726708
Epoch 20 Train C: 2.0113489830937956 Cl: 0.2228664112917141 Test C: 2.0098808445930754 Cl:0.2341614906832298
Epoch 30 Train C: 1.9719966684690193 Cl: 0.22296994167788245 Test C: 1.9697080725211058 Cl:0.2341614906832298
Epoch 40 Train C: 1.9480721644765864 Cl: 0.22296994167788245 Test C: 1.9448957248772862 Cl:0.2341614906832298
Epoch 50 Train C: 1.9330065240277852 Cl: 0.22296994167788245 Test C: 1.9290425941000573 Cl:0.2341614906832298
Epoch 60 Train C: 1.9234735905280886 Cl: 0.22296994167788245 Test C: 1.9187310586203696 Cl:0.2341614906832298
Epoch 70 Train C: 1.9177986013710349 Cl: 0.22296994167788245 Test C: 1.9124289105426997 Cl:0.2341614906832298
Epoch 80 Train C: 1.9146675456826223 Cl: 0.22296994167788245 Test C: 1.9089139179743388 Cl:0.2341614906832298
Epoch 90 Trai

Epoch 750 Train C: 1.8554688245872644 Cl: 0.250509024398661 Test C: 1.8485814054281855 Cl:0.2673913043478261
Epoch 760 Train C: 1.854206930740071 Cl: 0.2513717776167305 Test C: 1.8473064497714 Cl:0.268944099378882
Epoch 770 Train C: 1.852942086676128 Cl: 0.2524761017358595 Test C: 1.846019764856643 Cl:0.2704968944099379
Epoch 780 Train C: 1.8516590815836318 Cl: 0.25351140559754287 Test C: 1.8448506027292844 Cl:0.2711180124223602
Epoch 790 Train C: 1.8503661500614628 Cl: 0.25603064499430583 Test C: 1.8435972076674123 Cl:0.2736024844720497
Epoch 800 Train C: 1.849060798799295 Cl: 0.2562722158953653 Test C: 1.842325954866927 Cl:0.27391304347826084
Epoch 810 Train C: 1.8477406465708703 Cl: 0.259136556579356 Test C: 1.8410525816252423 Cl:0.27732919254658384
Epoch 820 Train C: 1.846417785641273 Cl: 0.2600338199261483 Test C: 1.839862166861986 Cl:0.27639751552795033
Epoch 830 Train C: 1.8450745034964504 Cl: 0.26051696172826727 Test C: 1.8385588072958325 Cl:0.27577639751552796
Epoch 840 Train 

Epoch 1490 Train C: 1.7834024357471487 Cl: 0.317217103219795 Test C: 1.7853197246865082 Cl:0.31490683229813665
Epoch 1500 Train C: 1.781291754910347 Cl: 0.3179418159229734 Test C: 1.7833460090145483 Cl:0.3167701863354037
Epoch 1510 Train C: 1.779863392844704 Cl: 0.31759671463574557 Test C: 1.7822319739207284 Cl:0.3170807453416149
Epoch 1520 Train C: 1.7778518088132327 Cl: 0.31732063360596335 Test C: 1.7811631735389677 Cl:0.31801242236024846
Epoch 1530 Train C: 1.775862331502365 Cl: 0.318632018497429 Test C: 1.7792856145192082 Cl:0.31894409937888196
Epoch 1540 Train C: 1.7751456039828961 Cl: 0.31873554888359734 Test C: 1.7786443087734722 Cl:0.31863354037267083
Epoch 1550 Train C: 1.7754223361286445 Cl: 0.3201849742899541 Test C: 1.7779228571303272 Cl:0.31894409937888196
Epoch 1560 Train C: 1.774065812191441 Cl: 0.3223246022707665 Test C: 1.7727348034886483 Cl:0.32298136645962733
Epoch 1570 Train C: 1.7942544008324217 Cl: 0.3021706870966629 Test C: 1.7816728324096611 Cl:0.309937888198757

Epoch 2230 Train C: 1.7329643903122214 Cl: 0.3354384511854229 Test C: 1.7449356010423802 Cl:0.3170807453416149
Epoch 2240 Train C: 1.7290515318248014 Cl: 0.3308140939365704 Test C: 1.7448395763555085 Cl:0.3133540372670807
Epoch 2250 Train C: 1.7379540956177282 Cl: 0.3257411050143217 Test C: 1.7545384832991755 Cl:0.31211180124223603
Epoch 2260 Train C: 1.73450648189053 Cl: 0.32743210132173795 Test C: 1.7514184967312705 Cl:0.3139751552795031
Epoch 2270 Train C: 1.7322993343856607 Cl: 0.32712151016323293 Test C: 1.749524791356207 Cl:0.3124223602484472
Epoch 2280 Train C: 1.7287573153222366 Cl: 0.331504296511026 Test C: 1.7459576637068945 Cl:0.3167701863354037
Epoch 2290 Train C: 1.7266781768154957 Cl: 0.33205645857059046 Test C: 1.7439083383457186 Cl:0.3177018633540373
Epoch 2300 Train C: 1.725513187984901 Cl: 0.3316768471546399 Test C: 1.742912688957986 Cl:0.3198757763975155
Epoch 2310 Train C: 1.7236732481275752 Cl: 0.33288470165993717 Test C: 1.7410223771582523 Cl:0.32111801242236027
E

Epoch 2970 Train C: 1.7196933675104267 Cl: 0.3345411878386306 Test C: 1.7354478314725028 Cl:0.31863354037267083
Epoch 2980 Train C: 1.7184670841001284 Cl: 0.3359561031162646 Test C: 1.7348849532629558 Cl:0.31925465838509315
Epoch 2990 Train C: 1.7118707684663519 Cl: 0.33895848431514647 Test C: 1.7288601944372708 Cl:0.3245341614906832
Epoch 3000 Train C: 1.706213184236924 Cl: 0.3408910515236222 Test C: 1.7247893241217322 Cl:0.3239130434782609
Epoch 3010 Train C: 1.699755863987498 Cl: 0.34330676053421677 Test C: 1.718964865659055 Cl:0.32608695652173914
Epoch 3020 Train C: 1.6994693787361135 Cl: 0.344687165683128 Test C: 1.7187190353170514 Cl:0.32857142857142857
Epoch 3030 Train C: 1.69892990397263 Cl: 0.34420402388100907 Test C: 1.718497454721752 Cl:0.3298136645962733
Epoch 3040 Train C: 1.6930626534460191 Cl: 0.34351382130655345 Test C: 1.7154388948910848 Cl:0.32919254658385094
Epoch 3050 Train C: 1.7130327416314575 Cl: 0.33550747144286847 Test C: 1.7362343040384296 Cl:0.325465838509316

...

Epoch 6680 Train C: 1.627304978663749 Cl: 0.3722952686613521 Test C: 1.6742473771153734 Cl:0.35838509316770184
Epoch 6690 Train C: 1.6369776007029446 Cl: 0.3717431066017876 Test C: 1.6849945956443326 Cl:0.3546583850931677
Epoch 6700 Train C: 1.623109048184903 Cl: 0.37916278427718536 Test C: 1.6710820027706685 Cl:0.3639751552795031
Epoch 6710 Train C: 1.6340080791883485 Cl: 0.3688442557890741 Test C: 1.6813948140797013 Cl:0.3521739130434783
Epoch 6720 Train C: 1.6504200166272822 Cl: 0.36473755047106327 Test C: 1.6924043238134676 Cl:0.3484472049689441
Epoch 6730 Train C: 1.636141237219576 Cl: 0.3700521102943714 Test C: 1.6779421286207956 Cl:0.35372670807453416
Epoch 6740 Train C: 1.628055625981465 Cl: 0.37215722814646096 Test C: 1.6712484773300829 Cl:0.35714285714285715
Epoch 6750 Train C: 1.621120567314494 Cl: 0.3745729371570556 Test C: 1.6666870801431923 Cl:0.3590062111801242
Epoch 6760 Train C: 1.625725902589861 Cl: 0.37277841046347104 Test C: 1.6729849416405822 Cl:0.3549689440993789


Epoch 7420 Train C: 1.6172891108698078 Cl: 0.3781619905442247 Test C: 1.6559766655072805 Cl:0.36366459627329195
Epoch 7430 Train C: 1.6084796615681856 Cl: 0.3835455706249784 Test C: 1.6511592465867033 Cl:0.3661490683229814
Epoch 7440 Train C: 1.6058162711035775 Cl: 0.3887566000621182 Test C: 1.6500101354417251 Cl:0.3695652173913043
Epoch 7450 Train C: 1.6132188741511146 Cl: 0.3801290678814232 Test C: 1.6530183676343435 Cl:0.36832298136645963
Epoch 7460 Train C: 1.6216947404755495 Cl: 0.37743727784104636 Test C: 1.6568088158575711 Cl:0.3630434782608696
Epoch 7470 Train C: 1.6269975671478665 Cl: 0.37163957621561927 Test C: 1.6623393421693557 Cl:0.36180124223602483
Epoch 7480 Train C: 1.6217309711541852 Cl: 0.3735376332953722 Test C: 1.6573006776445245 Cl:0.36335403726708076
Epoch 7490 Train C: 1.618096122437361 Cl: 0.37736825758360076 Test C: 1.6533148612503923 Cl:0.3658385093167702
Epoch 7500 Train C: 1.6338376197358544 Cl: 0.3728129205921938 Test C: 1.670222319755849 Cl:0.3506211180124

Epoch 8160 Train C: 1.6197790505152374 Cl: 0.37740276771232356 Test C: 1.6674760920359561 Cl:0.36086956521739133
Epoch 8170 Train C: 1.6098062669546702 Cl: 0.3803016185250371 Test C: 1.6590445861159429 Cl:0.3627329192546584
Epoch 8180 Train C: 1.604328346170831 Cl: 0.3824412465058495 Test C: 1.655235165879446 Cl:0.36490683229813664
Epoch 8190 Train C: 1.6068051912360575 Cl: 0.3806812299409877 Test C: 1.6581252329842193 Cl:0.3658385093167702
Epoch 8200 Train C: 1.6062855500314857 Cl: 0.38088829071332436 Test C: 1.6583128071502935 Cl:0.3661490683229814
Epoch 8210 Train C: 1.6040267120206542 Cl: 0.3821306553473445 Test C: 1.6569524022287878 Cl:0.36335403726708076
Epoch 8220 Train C: 1.615705082971987 Cl: 0.3789212133761259 Test C: 1.6691849728476467 Cl:0.3596273291925466
Epoch 8230 Train C: 1.6235584042390385 Cl: 0.3747454878006695 Test C: 1.678904813819205 Cl:0.3512422360248447
Epoch 8240 Train C: 1.6126863058478276 Cl: 0.3806467198122649 Test C: 1.6691785864918924 Cl:0.36024844720496896

Epoch 8900 Train C: 1.628834792688844 Cl: 0.37605687269213517 Test C: 1.6781264098531077 Cl:0.35838509316770184
Epoch 8910 Train C: 1.6124610813266649 Cl: 0.38092280084204716 Test C: 1.6630751499317973 Cl:0.36335403726708076
Epoch 8920 Train C: 1.6003479856411764 Cl: 0.3868585429823653 Test C: 1.6507123108575958 Cl:0.36801242236024845
Epoch 8930 Train C: 1.604127849985665 Cl: 0.3833730199813645 Test C: 1.6570590336264581 Cl:0.36366459627329195
Epoch 8940 Train C: 1.5937801241906806 Cl: 0.386755012596197 Test C: 1.6491543799402837 Cl:0.3667701863354037
Epoch 8950 Train C: 1.60999631764583 Cl: 0.3799220071090865 Test C: 1.6679378106225184 Cl:0.36118012422360246
Epoch 8960 Train C: 1.6116292449676006 Cl: 0.37919729440590816 Test C: 1.6709881569857505 Cl:0.35341614906832297
Epoch 8970 Train C: 1.599507554672561 Cl: 0.384235773199434 Test C: 1.6593676345649655 Cl:0.36149068322981365
Epoch 8980 Train C: 1.6099898611306556 Cl: 0.37778237912827417 Test C: 1.6677355922805877 Cl:0.35434782608695

Epoch 9640 Train C: 1.5828857155742124 Cl: 0.3902405355971978 Test C: 1.6418170795585039 Cl:0.37422360248447206
Epoch 9650 Train C: 1.5953603754609686 Cl: 0.3863754011802464 Test C: 1.655870067215199 Cl:0.365527950310559
Epoch 9660 Train C: 1.597933060370565 Cl: 0.38509852641750353 Test C: 1.660453277809928 Cl:0.36459627329192545
Epoch 9670 Train C: 1.59173377071988 Cl: 0.388791110190841 Test C: 1.6545692179144837 Cl:0.36335403726708076
Epoch 9680 Train C: 1.5842749392171365 Cl: 0.39096524830037616 Test C: 1.648190689442634 Cl:0.36366459627329195
Epoch 9690 Train C: 1.5987356753822877 Cl: 0.3796114159505815 Test C: 1.6605828265632885 Cl:0.35838509316770184
Epoch 9700 Train C: 1.6227055686894671 Cl: 0.3780239500293336 Test C: 1.6749212113690122 Cl:0.35745341614906834
Epoch 9710 Train C: 1.6005216221129765 Cl: 0.386444421437692 Test C: 1.6545794207166475 Cl:0.3652173913043478
Epoch 9720 Train C: 1.5877113118125832 Cl: 0.39037857611208887 Test C: 1.6434606886270922 Cl:0.3686335403726708
E

In [14]:
a.fit(X_train,y_train,lr=0.1,epochs=10000)

Epoch 0 Train C: 2.2847343798152178 Cl: 0.09811229595886392 Test C: 2.27947399866859 Cl:0.10186335403726708
Epoch 10 Train C: 1.919479946211839 Cl: 0.22296994167788245 Test C: 1.9136726226629244 Cl:0.2341614906832298
Epoch 20 Train C: 1.9145032471282102 Cl: 0.22296994167788245 Test C: 1.9086189798441933 Cl:0.2341614906832298


KeyboardInterrupt: 